In [ ]:
# pip install requests fhirclient jupyter

In [59]:
# Week2 – FHIR JSON & REST API (Jupyter Notebook)
# ------------------------------------------------

import requests
import json
from pathlib import Path
from IPython.display import display, JSON

# 설정
BASE_URL = "https://hapi.fhir.org/baseR4"
OUT_DIR = Path("./FHIR/week2")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1. Patient 데이터 가져오기
def fetch_patients(count=5):
    url = f"{BASE_URL}/Patient"
    params = {"_count": count}
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    return resp.json()

patient_bundle = fetch_patients(count=10)

# 2. JSON 원본 저장
with open(OUT_DIR / "patient_api.json", "w", encoding="utf-8") as f:
    json.dump(patient_bundle, f, indent=2, ensure_ascii=False)

# 3. 원본 JSON 미리보기
display(JSON(patient_bundle))

# 4. 깔끔하게 요약 출력
def summarize_patients(bundle):
    entries = bundle.get("entry", [])
    summary = []
    for e in entries:
        r = e.get("resource", {})
        if r.get("resourceType") != "Patient":
            continue
        # 이름
        name_text = "Unknown"
        names = r.get("name", [])
        if names:
            given = names[0].get("given", [])
            family = names[0].get("family", "")
            name_text = (" ".join(given) + " " + family).strip() or "Unknown"
        gender = r.get("gender", "Unknown")
        birth_date = r.get("birthDate", "Unknown")
        summary.append({"Name": name_text, "Gender": gender, "BirthDate": birth_date})
    return summary

from pandas import DataFrame
df_patients = DataFrame(summarize_patients(patient_bundle))
display(df_patients)


<IPython.core.display.JSON object>

""


In [61]:
import requests
import pandas as pd

BASE_URL = "https://hapi.fhir.org/baseR4/Patient"

def search_patients(params):
    """FHIR Patient 검색"""
    resp = requests.get(BASE_URL, params=params)
    resp.raise_for_status()
    return resp.json()

def bundle_to_dataframe(bundle):
    """FHIR Bundle → DataFrame 변환"""
    rows = []
    for e in bundle.get("entry", []):
        r = e.get("resource", {})
        if r.get("resourceType") != "Patient":
            continue
        rid = r.get("id", "Unknown")
        names = r.get("name", [])
        given = " ".join(names[0].get("given", [])) if names else ""
        family = names[0].get("family", "") if names else ""
        name = (given + " " + family).strip() or "Unknown"
        gender = r.get("gender", "Unknown")
        birth = r.get("birthDate", "Unknown")
        last_updated = r.get("meta", {}).get("lastUpdated", "Unknown")
        rows.append({
            "id": rid,
            "Name": name,
            "Gender": gender,
            "BirthDate": birth,
            "LastUpdated": last_updated
        })
    return pd.DataFrame(rows)

# 🔍 검색 조건 바꿔보기
search_params = {
    "name": "Jenny",                  # 이름에 'Jenny' 포함
    "birthdate": "1982-04-07",         # 생년월일
    "_sort": "-_lastUpdated",          # 최신순 정렬
    "_count": 5                        # 최대 5명
}

# 실행
bundle = search_patients(search_params)
df = bundle_to_dataframe(bundle)
df


,id,Name,Gender,BirthDate,LastUpdated
0,45710997,Jenny Moore,female,1982-04-07,2025-01-09T17:13:19.100+00:00
1,45710995,Jenny Moore,female,1982-04-07,2025-01-09T17:13:17.099+00:00
2,45710992,Jenny Moore,female,1982-04-07,2025-01-09T17:13:15.100+00:00
3,45710990,Jenny Moore,female,1982-04-07,2025-01-09T17:13:13.100+00:00
4,45710989,Jenny Moore,female,1982-04-07,2025-01-09T17:13:11.099+00:00


In [51]:
# fhir_patient_search.py
import requests

BASE_URL = "https://hapi.fhir.org/baseR4/Patient"

def search_patients(params):
    """FHIR Patient 검색"""
    resp = requests.get(BASE_URL, params=params)
    resp.raise_for_status()
    return resp.json()

def bundle_to_list(bundle):
    """FHIR Bundle → 환자 정보 리스트"""
    rows = []
    for e in bundle.get("entry", []):
        r = e.get("resource", {})
        if r.get("resourceType") != "Patient":
            continue
        rid = r.get("id", "Unknown")
        names = r.get("name", [])
        given = " ".join(names[0].get("given", [])) if names else ""
        family = names[0].get("family", "") if names else ""
        name = (given + " " + family).strip() or "Unknown"
        gender = r.get("gender", "Unknown")
        birth = r.get("birthDate", "Unknown")
        last_updated = r.get("meta", {}).get("lastUpdated", "Unknown")
        rows.append({
            "id": rid,
            "Name": name,
            "Gender": gender,
            "BirthDate": birth,
            "LastUpdated": last_updated
        })
    return rows

if __name__ == "__main__":
    # 검색 조건 (여기만 바꿔서 실행 가능)
    search_params = {
        "name": "Jenny",                  # 이름에 'Jenny' 포함
        "birthdate": "1982-04-07",         # 생년월일
        "_sort": "-_lastUpdated",          # 최신순 정렬
        "_count": 5                        # 최대 5명
    }

    bundle = search_patients(search_params)
    patients = bundle_to_list(bundle)

    print("\n검색 결과:")
    for p in patients:
        print(f"{p['id']} | {p['Name']} | {p['Gender']} | {p['BirthDate']} | {p['LastUpdated']}")



검색 결과:
45710997 | Jenny Moore | female | 1982-04-07 | 2025-01-09T17:13:19.100+00:00
45710995 | Jenny Moore | female | 1982-04-07 | 2025-01-09T17:13:17.099+00:00
45710992 | Jenny Moore | female | 1982-04-07 | 2025-01-09T17:13:15.100+00:00
45710990 | Jenny Moore | female | 1982-04-07 | 2025-01-09T17:13:13.100+00:00
45710989 | Jenny Moore | female | 1982-04-07 | 2025-01-09T17:13:11.099+00:00


In [63]:
# week2_fhir_patient_tool.py
"""
Week2 – FHIR JSON & REST API (Patients)
- requests로 Patient 검색/저장/요약
- 검색 파라미터(name, family, birthdate, _sort, _count) 지원
- 결과를 CSV로도 저장
- (옵션) fhirclient 예시 함수 포함

사용법:
  python week2_fhir_patient_tool.py --count 5 --sort -_lastUpdated
  python week2_fhir_patient_tool.py --name Jenny --birthdate 1982-04-07 --count 5
  python week2_fhir_patient_tool.py --family Kim --count 3 --out json
"""

import argparse
import json
from pathlib import Path
from typing import Dict, Any, List

import requests

BASE = "https://hapi.fhir.org/baseR4/Patient"
OUT_DIR = Path("./FHIR/week2")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def search_patients(params: Dict[str, Any]) -> Dict[str, Any]:
    resp = requests.get(BASE, params=params, timeout=30)
    resp.raise_for_status()
    return resp.json()

def bundle_to_rows(bundle: Dict[str, Any]) -> List[Dict[str, Any]]:
    rows = []
    for e in bundle.get("entry", []):
        r = e.get("resource", {})
        if r.get("resourceType") != "Patient":
            continue
        rid = r.get("id", "")
        meta = r.get("meta", {})
        last = meta.get("lastUpdated", "")
        # name 조합
        name = "Unknown"
        names = r.get("name", [])
        if names:
            given = " ".join(names[0].get("given", [])).strip()
            family = (names[0].get("family") or "").strip()
            full = (given + " " + family).strip()
            if full:
                name = full
        rows.append({
            "id": rid,
            "Name": name,
            "Gender": r.get("gender", "Unknown"),
            "BirthDate": r.get("birthDate", "Unknown"),
            "LastUpdated": last
        })
    return rows

def save_raw_json(bundle: Dict[str, Any], path: Path):
    path.write_text(json.dumps(bundle, indent=2, ensure_ascii=False), encoding="utf-8")

def save_csv(rows: List[Dict[str, Any]], path: Path):
    if not rows:
        path.write_text("", encoding="utf-8")
        return
    # 간단 CSV 저장
    headers = list(rows[0].keys())
    lines = [",".join(headers)]
    for r in rows:
        vals = [str(r.get(h, "")).replace(",", " ") for h in headers]
        lines.append(",".join(vals))
    path.write_text("\n".join(lines), encoding="utf-8")

def print_table(rows: List[Dict[str, Any]], max_rows: int = 20):
    if not rows:
        print("No Patient resources in the bundle.")
        return
    print("id | Name | Gender | BirthDate | LastUpdated")
    print("-"*80)
    for r in rows[:max_rows]:
        print(f"{r['id']} | {r['Name']} | {r['Gender']} | {r['BirthDate']} | {r['LastUpdated']}")

def main():
    ap = argparse.ArgumentParser(description="Week2 – FHIR Patient search tool")
    ap.add_argument("--name", type=str, help="name contains")
    ap.add_argument("--family", type=str, help="family(surname) contains")
    ap.add_argument("--given", type=str, help="given name contains")
    ap.add_argument("--birthdate", type=str, help='e.g. 1982-04-07 or ge1980-01-01')
    ap.add_argument("--count", type=int, default=5, help="_count (default 5)")
    ap.add_argument("--sort", type=str, default="-_lastUpdated", help="sort, e.g. -_lastUpdated")
    ap.add_argument("--out", choices=["json", "csv", "none"], default="json", help="save format")
    args = ap.parse_args()

    params = {"_count": args.count, "_sort": args.sort}
    if args.name: params["name"] = args.name
    if args.family: params["family"] = args.family
    if args.given: params["given"] = args.given
    if args.birthdate: params["birthdate"] = args.birthdate

    print("→ Searching with params:", params)
    bundle = search_patients(params)
    rows = bundle_to_rows(bundle)
    print_table(rows)

    # 저장
    if args.out != "none":
        if args.out == "json":
            raw_path = OUT_DIR / "patient_api.json"
            save_raw_json(bundle, raw_path)
            print(f"\nSaved raw JSON to: {raw_path}")
        elif args.out == "csv":
            csv_path = OUT_DIR / "patient_api.csv"
            save_csv(rows, csv_path)
            print(f"\nSaved CSV to: {csv_path}")

# (옵션) fhirclient 사용 예시
def demo_fhirclient():
    """
    pip install fhirclient 후 사용:
      from fhirclient import client
      from fhirclient.models import patient as fhir_patient
    """
    try:
        from fhirclient import client
        from fhirclient.models import patient as fhir_patient
    except Exception:
        print("Install fhirclient first: pip install fhirclient")
        return
    settings = {'app_id': 'my_app', 'api_base': 'https://hapi.fhir.org/baseR4'}
    smart = client.FHIRClient(settings=settings)
    patients = fhir_patient.Patient.where(struct={'_count': 3}).perform_resources(smart.server)
    for p in patients:
        name = "Unknown"
        if p.name and len(p.name) > 0:
            g = p.name[0].given or []
            fam = p.name[0].family or ""
            name = (" ".join(g) + " " + fam).strip() or "Unknown"
        gender = p.gender or "Unknown"
        bdate = p.birthDate.isostring if getattr(p, "birthDate", None) else "Unknown"
        print(f"{name} | {gender} | {bdate}")

if __name__ == "__main__":
    main()
    # demo_fhirclient()  # 필요하면 주석 해제


usage: ipykernel_launcher.py [-h] [--name NAME] [--family FAMILY]
                             [--given GIVEN] [--birthdate BIRTHDATE]
                             [--count COUNT] [--sort SORT]
                             [--out {json,csv,none}]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/cocoxoxo/Library/Jupyter/runtime/kernel-90c99b1e-0fd3-45a7-a341-fb8150a6239d.json


SystemExit: 2

/opt/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
